# Fusion des Données

Ce notebook fusionne toutes les sources de données extraites en un seul dataset prêt pour la modélisation.

## Sources de données

| Source | Features | Description |
|--------|----------|-------------|
| **Landsat V2** | ~20 | Bandes spectrales + indices (mean, std) |
| **TerraClimate V2** | ~34 | Climat avec lags et cumuls temporels |
| **ESA WorldCover** | ~11 | % occupation du sol |
| **SoilGrids** | 6 | Propriétés du sol (pH, argile, etc.) |
| **DEM** | 3 | Altitude, pente, orientation |
| **Water Type** | 1 | Type de milieu (rivière/lac) |

## Résultat attendu

- `merged_training.csv` : Dataset complet pour l'entraînement
- `merged_validation.csv` : Dataset complet pour la soumission

In [1]:
import pandas as pd
import numpy as np
import os

print("Imports OK!")

Imports OK!


---

## Étape 1 : Charger toutes les sources

In [2]:
# =============================================================================
# DONNÉES BRUTES (avec variables cibles)
# =============================================================================

# Training : contient les variables cibles
raw_training = pd.read_csv("../data/raw/water_quality_training_dataset.csv")
print(f"Raw Training : {raw_training.shape}")
print(f"Colonnes : {list(raw_training.columns)}")

# Validation : template de soumission
raw_validation = pd.read_csv("../data/raw/submission_template.csv")
print(f"\nRaw Validation : {raw_validation.shape}")

Raw Training : (9319, 6)
Colonnes : ['Latitude', 'Longitude', 'Sample Date', 'Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']

Raw Validation : (200, 6)


In [3]:
# =============================================================================
# CHARGER TOUTES LES FEATURES EXTRAITES
# =============================================================================

DATA_DIR = "../data/processed"

# Dictionnaire pour stocker les DataFrames
training_dfs = {}
validation_dfs = {}

# Liste des fichiers à charger
sources = {
    'landsat': ('landsat_features_training_v2.csv', 'landsat_features_validation_v2.csv'),
    'terraclimate': ('terraclimate_features_training_v2.csv', 'terraclimate_features_validation_v2.csv'),
    'worldcover': ('worldcover_features_training.csv', 'worldcover_features_validation.csv'),
    'soilgrids': ('soilgrids_features_training.csv', 'soilgrids_features_validation.csv'),
    'dem': ('dem_features_training.csv', 'dem_features_validation.csv'),
    'water_type': ('water_type_training.csv', 'water_type_validation.csv'),
}

print("Chargement des fichiers...\n")

for name, (train_file, val_file) in sources.items():
    train_path = os.path.join(DATA_DIR, train_file)
    val_path = os.path.join(DATA_DIR, val_file)
    
    if os.path.exists(train_path):
        training_dfs[name] = pd.read_csv(train_path)
        print(f"✅ {name} training : {training_dfs[name].shape}")
    else:
        print(f"❌ {name} training : fichier non trouvé")
    
    if os.path.exists(val_path):
        validation_dfs[name] = pd.read_csv(val_path)
        print(f"✅ {name} validation : {validation_dfs[name].shape}")
    else:
        print(f"❌ {name} validation : fichier non trouvé")
    
    print()

Chargement des fichiers...

✅ landsat training : (9319, 23)
✅ landsat validation : (200, 23)

✅ terraclimate training : (9319, 37)
✅ terraclimate validation : (200, 37)

✅ worldcover training : (9319, 14)
✅ worldcover validation : (200, 14)

✅ soilgrids training : (9319, 9)
✅ soilgrids validation : (200, 9)

✅ dem training : (9319, 6)
✅ dem validation : (200, 6)

✅ water_type training : (9319, 5)
✅ water_type validation : (200, 5)



### Inspecter les colonnes de chaque source

In [4]:
# Afficher les colonnes de chaque source
for name, df in training_dfs.items():
    print(f"\n{name.upper()} ({len(df.columns)} colonnes):")
    # Exclure les colonnes de jointure
    feature_cols = [c for c in df.columns if c not in ['Latitude', 'Longitude', 'Sample Date']]
    print(f"  Features: {feature_cols[:10]}{'...' if len(feature_cols) > 10 else ''}")


LANDSAT (23 colonnes):
  Features: ['blue', 'green', 'red', 'nir', 'swir16', 'swir22', 'blue_std', 'green_std', 'red_std', 'nir_std']...

TERRACLIMATE (37 colonnes):
  Features: ['pet', 'aet', 'ppt', 'ppt_lag1', 'ppt_lag2', 'ppt_lag3', 'ppt_sum4', 'ppt_mean4', 'ppt_anomaly', 'tmax']...

WORLDCOVER (14 colonnes):
  Features: ['lc_tree', 'lc_shrubland', 'lc_grassland', 'lc_cropland', 'lc_builtup', 'lc_bare', 'lc_snow', 'lc_water', 'lc_wetland', 'lc_mangroves']...

SOILGRIDS (9 colonnes):
  Features: ['soil_ph', 'soil_clay', 'soil_sand', 'soil_soc', 'soil_cec', 'soil_nitrogen']

DEM (6 colonnes):
  Features: ['elevation', 'slope', 'aspect']

WATER_TYPE (5 colonnes):
  Features: ['water_type', 'distance_to_river']


---

## Étape 2 : Fusionner les données

On fusionne sur les colonnes `Latitude`, `Longitude` et `Sample Date`.

In [5]:
def merge_all_sources(base_df, source_dfs, merge_keys=['Latitude', 'Longitude', 'Sample Date']):
    """
    Fusionne toutes les sources de données sur les clés spécifiées.
    
    Paramètres:
        base_df : DataFrame de base (avec les variables cibles pour training)
        source_dfs : dict de DataFrames à fusionner
        merge_keys : colonnes de jointure
    
    Retourne:
        DataFrame fusionné
    """
    merged = base_df.copy()
    
    for name, df in source_dfs.items():
        # Colonnes à ajouter (exclure les clés de jointure déjà présentes)
        cols_to_add = [c for c in df.columns if c not in merged.columns]
        
        # Déterminer les clés de jointure disponibles
        available_keys = [k for k in merge_keys if k in df.columns and k in merged.columns]
        
        if not available_keys:
            print(f"⚠️ {name}: pas de clés de jointure communes")
            continue
        
        if cols_to_add:
            # Sélectionner les colonnes pour la jointure
            df_subset = df[available_keys + cols_to_add].copy()
            
            # Fusionner
            merged = merged.merge(
                df_subset,
                on=available_keys,
                how='left'
            )
            print(f"✅ {name}: +{len(cols_to_add)} colonnes (jointure sur {available_keys})")
        else:
            print(f"⏭️ {name}: pas de nouvelles colonnes")
    
    return merged

In [6]:
# =============================================================================
# FUSION - TRAINING
# =============================================================================

print("Fusion des données TRAINING")
print("=" * 50)
print(f"Base : {raw_training.shape}")
print()

merged_training = merge_all_sources(raw_training, training_dfs)

print()
print(f"Résultat final : {merged_training.shape}")

Fusion des données TRAINING
Base : (9319, 6)

✅ landsat: +20 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])
✅ terraclimate: +34 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])
✅ worldcover: +11 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])
✅ soilgrids: +6 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])
✅ dem: +3 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])
✅ water_type: +2 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])

Résultat final : (9319, 82)


In [7]:
# =============================================================================
# FUSION - VALIDATION
# =============================================================================

print("Fusion des données VALIDATION")
print("=" * 50)
print(f"Base : {raw_validation.shape}")
print()

merged_validation = merge_all_sources(raw_validation, validation_dfs)

print()
print(f"Résultat final : {merged_validation.shape}")

Fusion des données VALIDATION
Base : (200, 6)

✅ landsat: +20 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])
✅ terraclimate: +34 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])
✅ worldcover: +11 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])
✅ soilgrids: +6 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])
✅ dem: +3 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])
✅ water_type: +2 colonnes (jointure sur ['Latitude', 'Longitude', 'Sample Date'])

Résultat final : (200, 82)


---

## Étape 3 : Vérification des données

In [8]:
# =============================================================================
# VÉRIFIER LES VALEURS MANQUANTES
# =============================================================================

print("Valeurs manquantes (Training)")
print("=" * 50)

missing = merged_training.isnull().sum()
missing_pct = (missing / len(merged_training) * 100).round(1)

# Afficher seulement les colonnes avec des valeurs manquantes
missing_cols = missing[missing > 0].sort_values(ascending=False)

if len(missing_cols) > 0:
    print(f"\n{len(missing_cols)} colonnes avec valeurs manquantes:")
    for col in missing_cols.index[:20]:  # Top 20
        print(f"  {col}: {missing[col]} ({missing_pct[col]}%)")
    if len(missing_cols) > 20:
        print(f"  ... et {len(missing_cols) - 20} autres")
else:
    print("Aucune valeur manquante !")

Valeurs manquantes (Training)

27 colonnes avec valeurs manquantes:
  soil_cec: 2918 (31.3%)
  soil_sand: 2918 (31.3%)
  soil_soc: 2918 (31.3%)
  soil_nitrogen: 2918 (31.3%)
  soil_clay: 2918 (31.3%)
  soil_ph: 2918 (31.3%)
  swir16: 100 (1.1%)
  nir: 100 (1.1%)
  blue: 100 (1.1%)
  green: 100 (1.1%)
  red: 100 (1.1%)
  swir16_std: 100 (1.1%)
  nir_std: 100 (1.1%)
  red_std: 100 (1.1%)
  green_std: 100 (1.1%)
  blue_std: 100 (1.1%)
  swir22: 100 (1.1%)
  swir22_std: 100 (1.1%)
  NDVI: 85 (0.9%)
  NDMI_std: 85 (0.9%)
  ... et 7 autres


In [9]:
# =============================================================================
# STATISTIQUES GLOBALES
# =============================================================================

print("Statistiques du dataset fusionné (Training)")
print("=" * 50)

print(f"\nDimensions : {merged_training.shape[0]} lignes x {merged_training.shape[1]} colonnes")

# Compter les types de colonnes
numeric_cols = merged_training.select_dtypes(include=[np.number]).columns
cat_cols = merged_training.select_dtypes(include=['object']).columns

print(f"\nColonnes numériques : {len(numeric_cols)}")
print(f"Colonnes catégorielles : {len(cat_cols)}")

# Variables cibles
target_cols = ['Alkalinity', 'Conductivity', 'Phosphorus']
print(f"\nVariables cibles :")
for col in target_cols:
    if col in merged_training.columns:
        print(f"  {col}: mean={merged_training[col].mean():.2f}, std={merged_training[col].std():.2f}")

Statistiques du dataset fusionné (Training)

Dimensions : 9319 lignes x 82 colonnes

Colonnes numériques : 80
Colonnes catégorielles : 2

Variables cibles :


In [10]:
# =============================================================================
# LISTE DES FEATURES PAR CATÉGORIE
# =============================================================================

# Colonnes de base (pas des features)
base_cols = ['Latitude', 'Longitude', 'Sample Date', 'Alkalinity', 'Conductivity', 'Phosphorus']

# Identifier les features par préfixe/source
feature_groups = {
    'Landsat': [c for c in merged_training.columns if any(x in c.lower() for x in ['blue', 'green', 'red', 'nir', 'swir', 'ndvi', 'ndwi', 'mndwi', 'ndti'])],
    'TerraClimate': [c for c in merged_training.columns if any(x in c.lower() for x in ['ppt', 'tmax', 'tmin', 'soil', 'def', 'vpd', 'aet', 'pet', 'srad', 'swe'])],
    'WorldCover': [c for c in merged_training.columns if c.startswith('lc_')],
    'SoilGrids': [c for c in merged_training.columns if c.startswith('soil_')],
    'DEM': [c for c in merged_training.columns if c in ['elevation', 'slope', 'aspect']],
    'WaterType': [c for c in merged_training.columns if c == 'water_type'],
}

print("Features par source")
print("=" * 50)

total_features = 0
for group, cols in feature_groups.items():
    print(f"\n{group} ({len(cols)} features):")
    if len(cols) <= 10:
        print(f"  {cols}")
    else:
        print(f"  {cols[:5]} ... {cols[-5:]}")
    total_features += len(cols)

print(f"\n{'=' * 50}")
print(f"TOTAL : {total_features} features")

Features par source

Landsat (18 features):
  ['blue', 'green', 'red', 'nir', 'swir16'] ... ['NDWI', 'MNDWI', 'NDVI_std', 'NDWI_std', 'MNDWI_std']

TerraClimate (38 features):
  ['pet', 'aet', 'ppt', 'ppt_lag1', 'ppt_lag2'] ... ['soil_clay', 'soil_sand', 'soil_soc', 'soil_cec', 'soil_nitrogen']

WorldCover (11 features):
  ['lc_tree', 'lc_shrubland', 'lc_grassland', 'lc_cropland', 'lc_builtup'] ... ['lc_snow', 'lc_water', 'lc_wetland', 'lc_mangroves', 'lc_moss']

SoilGrids (12 features):
  ['soil_lag1', 'soil_lag2', 'soil_lag3', 'soil_sum4', 'soil_mean4'] ... ['soil_clay', 'soil_sand', 'soil_soc', 'soil_cec', 'soil_nitrogen']

DEM (3 features):
  ['elevation', 'slope', 'aspect']

WaterType (1 features):
  ['water_type']

TOTAL : 83 features


---

## Étape 4 : Nettoyage (optionnel)

In [11]:
# =============================================================================
# SUPPRIMER LES COLONNES INUTILES
# =============================================================================

# Colonnes WorldCover toujours à 0 (pas présentes en Afrique du Sud)
cols_to_drop = ['lc_snow', 'lc_mangroves', 'lc_moss']

# Vérifier si ces colonnes existent et sont bien à 0
for col in cols_to_drop:
    if col in merged_training.columns:
        if merged_training[col].sum() == 0:
            print(f"Suppression de {col} (toujours 0)")
            merged_training = merged_training.drop(columns=[col])
            if col in merged_validation.columns:
                merged_validation = merged_validation.drop(columns=[col])

print(f"\nDimensions après nettoyage : {merged_training.shape}")

Suppression de lc_snow (toujours 0)
Suppression de lc_mangroves (toujours 0)
Suppression de lc_moss (toujours 0)

Dimensions après nettoyage : (9319, 79)


---

## Étape 5 : Sauvegarder les datasets fusionnés

In [12]:
# =============================================================================
# SAUVEGARDER LES FICHIERS
# =============================================================================

OUTPUT_DIR = "../data/processed"

# Training
train_path = os.path.join(OUTPUT_DIR, 'merged_training.csv')
merged_training.to_csv(train_path, index=False)
print(f"✅ Sauvegardé : {train_path}")
print(f"   {merged_training.shape[0]} lignes x {merged_training.shape[1]} colonnes")

# Validation
val_path = os.path.join(OUTPUT_DIR, 'merged_validation.csv')
merged_validation.to_csv(val_path, index=False)
print(f"\n✅ Sauvegardé : {val_path}")
print(f"   {merged_validation.shape[0]} lignes x {merged_validation.shape[1]} colonnes")

✅ Sauvegardé : ../data/processed\merged_training.csv
   9319 lignes x 79 colonnes

✅ Sauvegardé : ../data/processed\merged_validation.csv
   200 lignes x 79 colonnes


In [13]:
# Aperçu final
print("Aperçu du dataset fusionné (Training)")
display(merged_training.head())

Aperçu du dataset fusionné (Training)


,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus,blue,green,red,nir,...,soil_clay,soil_sand,soil_soc,soil_cec,soil_nitrogen,elevation,slope,aspect,water_type,distance_to_river
0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0,9824.479167,11602.008333,12700.020833,13351.962500,...,69.0,838.0,546.0,166.0,141.0,192.663025,11.798665,299.497650,river,46.592505
1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0,8511.409524,9483.828571,8980.290476,19672.442857,...,302.0,540.0,161.0,200.0,200.0,1527.916626,2.923243,109.644104,river,69.035778
2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0,9073.283898,9939.313559,11237.631356,13288.343220,...,260.0,627.0,180.0,218.0,144.0,1473.671143,1.366939,134.574402,river,123.538285
3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0,9790.723214,10862.504464,10937.803571,15800.040179,...,264.0,616.0,162.0,195.0,129.0,1347.080688,3.807301,310.537842,river,20.518409
4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0,8812.910714,9734.102679,9446.839286,16401.950893,...,245.0,637.0,150.0,186.0,132.0,1357.651001,1.690194,224.774612,river,46.165896


---

## Résumé

**Fichiers créés :**

| Fichier | Description |
|---------|-------------|
| `merged_training.csv` | Dataset complet pour l'entraînement |
| `merged_validation.csv` | Dataset complet pour la soumission |

**Prochaine étape :**
- Entraîner le modèle avec toutes les nouvelles features
- Comparer R² avant (~0.41) vs après
- Tester XGBoost / LightGBM